In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import glob

from sklearn.metrics import confusion_matrix
import itertools

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPool1D
from keras.initializers import random_uniform
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler as SS

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from matplotlib import pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,Dense,LSTM,Dropout,Flatten,BatchNormalization,Conv1D,GlobalMaxPooling1D,MaxPooling1D
from keras.optimizers import  SGD
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.optimizers import Adam
from keras import regularizers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing import sequence
from keras.layers import SimpleRNN
#from hyperas.distributions import uniform

from keras.utils.np_utils import to_categorical
from keras import regularizers
import pandas as pd
import string
import numpy as np
import matplotlib.pyplot as plt

# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')


### Importing the dataset

In [2]:
dataset = pd.read_csv(r'...') # Use your path

In [3]:
dataset.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Label
0,0,1,24,9,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,1,24,9,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,1,24,9,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,0,1,24,9,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,0,1,24,9,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.


In [4]:
dataset.shape

(1945562, 40)

In [5]:
# Splitting dataset into features and labels.
labels = dataset['Label']
features = dataset.loc[:, dataset.columns != 'Label'].astype('float64')

In [6]:
# For scaling the data, we use RobustScaler class from sklearn.

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

In [7]:
scaler = RobustScaler()
scaler.fit(features)

features = scaler.transform(features)

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
LE = LabelEncoder()

LE.fit(labels)
labels = LE.transform(labels)

In [10]:
# Labels have been replaced with integers.

np.unique(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [11]:
# Checking that encoding reversal works.

d = LE.inverse_transform(labels)
d = pd.Series(d)
d.unique()

array(['normal.', 'smurf.', 'neptune.', 'satan.', 'ipsweep.',
       'portsweep.', 'back.', 'nmap.', 'warezclient.', 'pod.',
       'teardrop.', 'guess_passwd.', 'land.', 'buffer_overflow.',
       'warezmaster.', 'multihop.', 'rootkit.', 'loadmodule.', 'phf.',
       'ftp_write.'], dtype=object)

### Performing cross validation for neural network with 1 layer

In [12]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [29]:
# Function used to create the model needed for KerasClassifier
def create_model():
    # Compile model
    dnn1= Sequential()
    dnn1.add(Dense(256,input_dim=39 ,activation='relu'))
    dnn1.add(Dropout(0.1))
    dnn1.add(Dense(20, activation='softmax'))
    
    # Compile model
    dnn1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return dnn1

In [30]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [31]:
# create model
model = KerasClassifier( build_fn=create_model, epochs= 30, batch_size=64, verbose=0)

In [32]:
# 3-fold cross validation
# StratifiedKFold is a variation of Kfold that first shuffles the data with a given random state
kfold = StratifiedKFold(n_splits= 3, shuffle=True, random_state=seed)

In [33]:
# Printing the mean accuracy value for the 3 folds
results = cross_val_score(model, features, labels, cv=kfold,scoring='accuracy', error_score="raise")
print(results.mean())
results

0.9996571684529582


array([0.99966077, 0.9996839 , 0.99962684])

### Performing cross validation for neural network with 2 layers

In [34]:
# Function used to create the model needed for KerasClassifier
def create_model_dnn2():
    # Compile model
    model= Sequential()
    model.add(Dense(256,input_dim=39 ,activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(20, activation='softmax'))
    
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [35]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [36]:
# create model
model = KerasClassifier( build_fn=create_model_dnn2, epochs= 30, batch_size=64, verbose=0)

In [37]:
# 3-fold cross validation
# StratifiedKFold is a variation of Kfold that first shuffles the data with a given random state
kfold = StratifiedKFold(n_splits= 3, shuffle=True, random_state=seed)

In [38]:
# Printing the mean accuracy value for the 3 folds
results = cross_val_score(model, features, labels, cv=kfold,scoring='accuracy', error_score="raise")
print(results.mean())
results

0.9996695042423985


array([0.99964843, 0.99967773, 0.99968235])

### Performing cross validation for neural network with 3 layers

In [39]:
# Function used to create the model needed for KerasClassifier
def create_model_dnn3():
    # Compile model
    model= Sequential()
    model.add(Dense(256,input_dim=39 ,activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(20, activation='softmax'))
    
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [40]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [41]:
# create model
model = KerasClassifier( build_fn=create_model_dnn3, epochs= 30, batch_size=64, verbose=0)

In [42]:
# 3-fold cross validation
# StratifiedKFold is a variation of Kfold that first shuffles the data with a given random state
kfold = StratifiedKFold(n_splits= 3, shuffle=True, random_state=seed)

In [43]:
# Printing the mean accuracy value for the 3 folds
results = cross_val_score(model, features, labels, cv=kfold,scoring='accuracy', error_score="raise")
print(results.mean())
results

0.9993158788482903


array([0.99934004, 0.99940788, 0.99919972])

### Performing cross validation for recurrent neural network with 1 layer

In [50]:
# Function used to create the model needed for KerasClassifier
def create_model_rnn1():
    # create model
    SEED = 1040941203
    hidden_initializer = random_uniform(seed=SEED)

    # Initialising the RNN
    rnn1 = Sequential()

    # Adding the first RNN layer and some Dropout regularisation
    rnn1.add(SimpleRNN(units = 64,input_shape=(39, 1), activation='relu', return_sequences = True,))
    rnn1.add(Dropout(0.3))

    # Adding the output layer
    rnn1.add(Flatten())
    rnn1.add(Dense(64, input_dim=39, kernel_initializer=hidden_initializer))
    rnn1.add(LeakyReLU(alpha=0.1))
    rnn1.add(Dense(32))
    rnn1.add(LeakyReLU(alpha=0.1))
    rnn1.add(Dense(units = 20, activation='softmax'))
    
    # Compile model
    rnn1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return rnn1


In [51]:
# create model
rnn1 = KerasClassifier( build_fn=create_model_rnn1, epochs= 30, batch_size=512, verbose=0)

In [52]:
# 3-fold cross validation
Three_kfold = StratifiedKFold(n_splits= 3, shuffle=True, random_state=seed)

In [53]:
ft_train = np.array(features).reshape(features.shape[0], features.shape[1], 1)
ft_labels= to_categorical(labels)

In [55]:
# Printing the mean accuracy value for the 3 folds
results_rnn1 = cross_val_score(rnn1, ft_train, labels, cv=Three_kfold,scoring='accuracy', error_score="raise")
print(results_rnn1.mean())

0.9995934337825799


### Performing cross validation for recurrent neural network with 2 layer

In [ ]:
# Function used to create the model needed for KerasClassifier
def create_model_rnn2():
    # create model
    SEED = 1040941203
    hidden_initializer = random_uniform(seed=SEED)

    # Initialising the RNN
    rnn2 = Sequential()

    # Adding the first RNN layer and some Dropout regularisation
    rnn2.add(SimpleRNN(units = 32,activation='relu', return_sequences = True,  input_shape = (39,1)))
    rnn2.add(Dropout(0.3))

    # Adding the first RNN layer and some Dropout regularisation
    rnn2.add(SimpleRNN(units = 32,activation='relu', return_sequences = True,  ))
    rnn2.add(Dropout(0.3))

    # Adding the output layer
    rnn2.add(Flatten())
    rnn2.add(Dense(64, input_dim=39, kernel_initializer=hidden_initializer))
    rnn2.add(LeakyReLU(alpha=0.1))
    rnn2.add(Dense(32))
    rnn2.add(LeakyReLU(alpha=0.1))
    rnn2.add(Dense(units = 20, activation='softmax'))
    
    # Compile model
    rnn2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return rnn2


In [ ]:
# create model
rnn2 = KerasClassifier( build_fn=create_model_rnn2, epochs= 30, batch_size=1000, verbose=0)

In [ ]:
# 3-fold cross validation
Three_kfold = StratifiedKFold(n_splits= 3, shuffle=True, random_state=seed)

In [ ]:
ft_train = np.array(features).reshape(features.shape[0], features.shape[1], 1)
ft_labels= to_categorical(labels)

In [ ]:
# Printing the mean accuracy value for the 3 folds
results_rnn2 = cross_val_score(rnn2, ft_train, labels, cv=Three_kfold,scoring='accuracy', error_score="raise")
print(results_rnn2.mean())

### Performing cross validation for recurrent neural network with 3 layer

In [ ]:
# Function used to create the model needed for KerasClassifier
def create_model_rnn3():
    # create model
    SEED = 1040941203
    hidden_initializer = random_uniform(seed=SEED)

    # Initialising the RNN
    rnn3 = Sequential()

    # Adding the first RNN layer and some Dropout regularisation
    rnn3.add(SimpleRNN(units = 32,activation='relu', return_sequences = True,  input_shape = (39,1)))
    rnn3.add(Dropout(0.3))

    # Adding the first RNN layer and some Dropout regularisation
    rnn3.add(SimpleRNN(units = 32,activation='relu', return_sequences = True,  ))
    rnn3.add(Dropout(0.3))
    
    # Adding a third RNN layer and some Dropout regularisation
    rnn3.add(SimpleRNN(units = 32,activation='relu', return_sequences = True))
    rnn3.add(Dropout(0.1))

    # Adding the output layer
    rnn3.add(Flatten())
    rnn3.add(Dense(64, input_dim=39, kernel_initializer=hidden_initializer))
    rnn3.add(LeakyReLU(alpha=0.1))
    rnn3.add(Dense(32))
    rnn3.add(LeakyReLU(alpha=0.1))
    rnn3.add(Dense(units = 20, activation='softmax'))
    
    # Compile model
    rnn3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return rnn3


In [ ]:
# create model
rnn3 = KerasClassifier( build_fn=create_model_rnn3, epochs= 30, batch_size=1000, verbose=0)

In [ ]:
# 3-fold cross validation
Three_kfold = StratifiedKFold(n_splits= 3, shuffle=True, random_state=seed)

In [ ]:
ft_train = np.array(features).reshape(features.shape[0], features.shape[1], 1)
ft_labels= to_categorical(labels)

In [ ]:
# Printing the mean accuracy value for the 3 folds
results_rnn3 = cross_val_score(rnn3, ft_train, labels, cv=Three_kfold,scoring='accuracy', error_score="raise")
print(results_rnn3.mean())

### Performing cross validation for convolutional neural network with 1 layer

In [ ]:
# 3-fold cross validation
Three_kfold = StratifiedKFold(n_splits= 3, shuffle=True, random_state=seed)

In [ ]:
y_train= to_categorical(labels)
X_train = np.array(features).reshape(features.shape[0], features.shape[1], 1)

In [ ]:
# Function used to create the model needed for KerasClassifier
def create_model_cnn1():
    # create model
    SEED = 1040941203
    hidden_initializer = random_uniform(seed=SEED)

    cnn1 = Sequential()
    cnn1.add(Conv1D(64, 3,input_shape=(39, 1) ))
    cnn1.add(LeakyReLU(alpha=0.1))
    cnn1.add(MaxPooling1D(pool_size=2))
    cnn1.add(Dropout(0.3))

    cnn1.add(Flatten())
    cnn1.add(Dense(64, input_dim=39, kernel_initializer=hidden_initializer))
    cnn1.add(LeakyReLU(alpha=0.1))

    cnn1.add(Dense(20, activation='softmax'))
    
    # Compile model
    cnn1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return cnn1


In [ ]:
# create model
cnn1 = KerasClassifier( build_fn=create_model_cnn1, epochs= 30, batch_size=256, verbose=0)

In [ ]:
# Printing the mean accuracy value for the 5 folds
results_cnn1 = cross_val_score(cnn1, X_train, labels, cv=kfold,scoring='accuracy', error_score="raise")
print(results_cnn1.mean())
results_cnn1

### Performing cross validation for convolutional neural network with 2 layer

In [ ]:
# Function used to create the model needed for KerasClassifier
def create_model_cnn2():
    # create model
    SEED = 1040941203
    hidden_initializer = random_uniform(seed=SEED)

    cnn2 = Sequential()
    cnn2.add(Conv1D(64, 3,input_shape=(39, 1) ))
    cnn2.add(LeakyReLU(alpha=0.1))
    cnn2.add(MaxPooling1D(pool_size=2))
    cnn2.add(Dropout(0.3))

    cnn2.add(Conv1D(32, 1))
    cnn2.add(LeakyReLU(alpha=0.1))
    cnn2.add(MaxPooling1D(pool_size=2))
    cnn2.add(Dropout(0.3))

    cnn2.add(Flatten())
    cnn2.add(Dense(64, input_dim=39, kernel_initializer=hidden_initializer))
    cnn2.add(LeakyReLU(alpha=0.1))

    cnn2.add(Dense(20, activation='softmax'))
    
    # Compile model
    cnn2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return cnn2


In [ ]:
# create model
cnn2 = KerasClassifier(build_fn=create_model_cnn2, epochs= 30, batch_size=512, verbose=0)

In [ ]:
# Printing the mean accuracy value for the 3 folds
results_cnn2 = cross_val_score(cnn2, X_train, labels, cv=Three_kfold,scoring='accuracy', error_score="raise")
print(results_cnn2.mean())
results_cnn2

### Performing cross validation for convolutional neural network with 3 layer

In [ ]:
# Function used to create the model needed for KerasClassifier
def create_model_cnn3():
    # create model
    SEED = 1040941203
    hidden_initializer = random_uniform(seed=SEED)

    cnn3 = Sequential()
    cnn3.add(Conv1D(64, 3,input_shape=(39, 1) ))
    cnn3.add(LeakyReLU(alpha=0.1))
    cnn3.add(MaxPooling1D(pool_size=2))
    cnn3.add(Dropout(0.3))

    cnn3.add(Conv1D(64, 1))
    cnn3.add(LeakyReLU(alpha=0.1))
    cnn3.add(MaxPooling1D(pool_size=2))
    cnn3.add(Dropout(0.3))

    cnn3.add(Conv1D(64, 1))
    cnn3.add(LeakyReLU(alpha=0.1))
    cnn3.add(MaxPooling1D(pool_size=2))
    cnn3.add(Dropout(0.3))


    cnn3.add(Flatten())
    cnn3.add(Dense(64, input_dim=39, kernel_initializer=hidden_initializer))
    cnn3.add(LeakyReLU(alpha=0.1))

    cnn3.add(Dense(20, activation='softmax'))
    
    # Compile model
    cnn3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return cnn3


In [ ]:
# create model
cnn3 = KerasClassifier( build_fn=create_model_cnn3, epochs= 30, batch_size=512, verbose=0)

In [ ]:
# Printing the mean accuracy value for the 3 folds
results_cnn3 = cross_val_score(cnn3, X_train, labels, cv=Three_kfold,scoring='accuracy', error_score="raise")
print(results_cnn3.mean())
results_cnn3